---

# Model deployment

## Importar librerias

In [1]:
import mlflow
import sys
import os

sys.path.append("../../")
from ML.utils.mlflow_flow import set_tracking
from dotenv import load_dotenv

load_dotenv()
ENDPOINT_URL = os.getenv("ENDPOINT_URL")
DAGSHUB_PUBLIC_ACCESS_KEY = os.getenv("MLFLOW_TRACKING_TOKEN")
MODEL_NAME = os.getenv("MODEL_NAME")

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/yosesotomayor/retoCasas_v2.mlflow"
os.environ["MLFLOW_REGISTRY_URI"] = "https://dagshub.com/yosesotomayor/retoCasas_v2.mlflow"



---

In [25]:
c = MlflowClient()
versions = c.search_model_versions("name='elnet_lgbm'")

# filtra solo los que tienen rmse en tags
with_rmse = [mv for mv in versions if "rmse" in mv.tags]
if not with_rmse:
    raise RuntimeError("Ninguna versión tiene tag 'rmse'. Agrega rmse al registrar tus modelos.")

best = min(with_rmse, key=lambda mv: float(mv.tags["rmse"]))

print("Champion version:", best.version, "rmse:", best.tags['rmse'])
model = mlflow.pyfunc.load_model(f"models:/elnet_lgbm/{best.version}")

RuntimeError: Ninguna versión tiene tag 'rmse'. Agrega rmse al registrar tus modelos.

## Descargar modelo

/var/folders/gw/6t_48cb11cq3wt9bjf_ns5p40000gn/T/ipykernel_86389/2561229697.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  version = c.get_latest_versions(MODEL_NAME)[0].version


KeyboardInterrupt: 

In [15]:
from mlflow.tracking import MlflowClient

sys.path.append("../models/ensemble_elnet_lgbm")

STAGE = 'Production'
print("TRACKING:", os.getenv("MLFLOW_TRACKING_URI"))
print("REGISTRY:", os.getenv("MLFLOW_REGISTRY_URI"))
set_tracking(ENDPOINT_URL)

c = MlflowClient()
runs = c.search_runs(experiment_ids=["0"], order_by=["metrics.rmse ASC"], max_results=1)
best_run_id = runs[0]
print(best_run_id)

model = mlflow.pyfunc.load_model(f"models:/{MODEL_NAME}")

TRACKING: https://dagshub.com/yosesotomayor/retoCasas_v2.mlflow
REGISTRY: https://dagshub.com/yosesotomayor/retoCasas_v2.mlflow


MlflowException: API request to endpoint /api/2.0/mlflow/runs/search failed with error code 403 != 200. Response body: ''

---